In [1]:
# -*- coding: utf-8 -*-
"""
Vindhya Balasubramanyam
HW 7 - Bread Basket Analysis
"""
import os
import pandas as pd


# Declare variables required
DASHES = '-' * 10
TABS = '\t' * 8
input_dir = r'.'
    
# pd options
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_columns', 12)
pd.set_option('display.expand_frame_repr', False)
pd.options.display.float_format = '{:,.2f}'.format
    
##set filenames
input_dir = os.getcwd()
root_dir = os.getcwd()

bread_file='BreadBasket_DMS_output'
bread_path_file = os.path.join(input_dir, bread_file+'.csv')

# Read file 
try:
   bread_orig_df = pd.read_csv(bread_path_file)
except Exception as e:
  print(e)
  print('Failed to read input files', bread_file)

print(f'\nBread Basket dataset has been read into a data frame:  {bread_orig_df.head()}')

# Reduce data frame to columns needed
bread_df = bread_orig_df[['Weekday', 'Period', 'Hour', 'Transaction', 'Item', 'Item_Price']]

# Compute busiest hour
print(f"\tThe busiest timeframes for transactions are as follows.")
busy_hour = bread_df.groupby(by='Hour')['Transaction'].nunique().idxmax()
print(f'\t-Hour: {busy_hour}')
# Compute busiest day of week
busy_day = bread_df.groupby(by='Weekday')['Transaction'].nunique().idxmax()
print(f'\t-Day of Week: {busy_day}')
# Compute busiest period
busy_period = bread_df.groupby(by='Period')['Transaction'].nunique().idxmax()
print(f'\t-Period: {busy_period}')
    
# Compute the most profitable hour
print(f"\tThe most profitable times for highest revenue are as follows.")
profit_hour = bread_df.groupby(by='Hour')['Item_Price'].sum().idxmax()
print(f'\t-Hour: {profit_hour}')
# Compute the most profitable day of week
profit_day = bread_df.groupby(by='Weekday')['Item_Price'].sum().idxmax()
print(f'\t-Day of Week: {profit_day}')
# Compute the most profitable period
profit_period = bread_df.groupby(by='Period')['Item_Price'].sum().idxmax()
print(f'\t-Period: {profit_period}')
       

# Compute and print the most and least popular items
popular_df = bread_df.groupby(by='Item')['Transaction'].count()
print(f'\tMost popular item is: {popular_df.idxmax()}' f' ({popular_df[popular_df.idxmax()]} units sold)')
print(f'\tLeast popular item is: {popular_df.idxmin()}'
f' ({popular_df[popular_df.idxmin()]} unit sold)')

# Number of transactions/(week)day: 50
trans_limit = 50
# Compute number of unique transactions per day
transactions_df = bread_df.groupby(by='Weekday')['Transaction'].nunique().reset_index()
transactions_df['Num_Baristas_per_day'] = transactions_df.apply(lambda row: \
            round(row.Transaction/trans_limit), axis=1)
        
transactions_df.columns=['Weekday','Total_Transactions', 'Baristas_ per_Day']
        
print(f'\tNumber of baristas required on each weekday given the limit of transactions a barista can handle is: \n\n {transactions_df}')

# Group items into categories
food_list= ['Bread', 'Scandinavian', 'Jam', 'Cookies',\
            'Muffin', 'Pastry', 'Medialuna', 'Tartine',  'Farm House',\
            'Fudge', 'Frittata', 'Soup', 'Pick and Mix Bowls', 'Cake',\
            'Mighty Protein', 'Chicken sand',  'Focaccia', 'Sandwich', \
            'Alfajores', 'Eggs', 'Brownie', 'Honey',  'Granola', 'Fairy Doors', \
            'Empanadas', 'Bowl Nic Pitt', 'Bread Pudding', 'Truffles', 'Chimichurri Oil',\
            'Bacon', 'Spread', 'Kids biscuit', 'Caramel bites',\
            'Jammie Dodgers', 'Tiffin', 'Olum & polenta', 'Polenta',\
            'Lemon and coconut', 'Toast', 'Scone', 'Crepes', 'Vegan mincepie',\
            'Bare Popcorn', 'Muesli', 'Crisps', 'Gingerbread syrup',\
            'Panatone', 'Brioche and salami', 'Salad',\
            'Chicken Stew', 'Spanish Brunch', 'Raspberry shortbread sandwich',\
            'Extra Salami or Feta','Duck egg', 'Baguette', 'Vegan Feast',\
            'Chocolates', 'Coffee granules ', 'Cherry me Dried fruit', 'Raw bars', 'Dulce de Leche', 'Tacos/Fajita']

drink_list =['Hot chocolate', 'Coffee', 'Tea', 'Mineral water', 'Juice', 'Smoothies', 'Coke', 'My-5 Fruit Shoot']

# Compute number of unique transactions per day
categories_df = bread_df.copy()
categories_df['Category']=categories_df.apply(lambda row: \
            'food' if row['Item'] in food_list else\
            ('drink' if row['Item'] in drink_list else 'misc'), axis=1)
        
# Compute the average prices
avg_price_df = categories_df.groupby(by='Category')['Item_Price'].mean()

print(f"\tThe average price of food items is: ${avg_price_df.at['food']:,.2f}")
print(f"\tThe average price of drink items is: ${avg_price_df.at['drink']:,.2f}")

# Compute number of items sold in category
sum_df = categories_df.groupby(by='Category')['Item_Price'].sum()

# Compute number of unique transactions per day
if sum_df.at['drink'] > sum_df.at['food']:
   print(f'\tThe bakery makes more money selling drinks.\n'
         f'\tTotal revenue from drinks: ${sum_df.at["drink"]:,.2f}'
        f'\n\tTotal revenue from food: ${sum_df.at["food"]:,.2f}')
else:
   print(f'\n\tThe bakery makes more money selling food.\n'
         f'\n\tTotal revenue from food: ${sum_df.at["food"]:,.2f}')
            

print(f"\tThe 5 MOST popular items for each day of the week:")
day_agg_df = pd.DataFrame(bread_df.groupby(by=['Weekday','Item'])['Transaction'].count())
most_popular_df = day_agg_df.groupby(by='Weekday')['Transaction'].nlargest(5)
print(f'{most_popular_df}')
print("\tAs you can see from the list above, the popular items varies across weekdays.")

print(f'\tThe 5 LEAST popular items for each day of the week:')
least_popular_df = day_agg_df.groupby(by='Weekday')['Transaction'].nsmallest(5)
print(f'{least_popular_df}')
print("\tAs you can see from the list above, the least popular items varies across weekdays.")        
        

# Compute number of drinks in each trasaction
drink_df = pd.DataFrame(categories_df.groupby(by=['Transaction','Category'])\
            .count()['Item']).filter(like = 'drink', axis=0)
        
# Average drink/transaction:
avg_num_drinks = float(drink_df.sum()/len(categories_df['Transaction'].unique()))

#print(f"\tNumber of drinks in each transaction:\n {drink_df}")
print(f"\tAverage number of drinks per transaction  {avg_num_drinks:,.2f}")


Bread Basket dataset has been read into a data frame:     Year  Month  Day Weekday   Period  Hour  Min  Sec  Transaction           Item  Item_Price
0  2016     10   30  Sunday  morning     9   58   11            1          Bread        3.01
1  2016     10   30  Sunday  morning    10    5   34            2   Scandinavian       10.38
2  2016     10   30  Sunday  morning    10    5   34            2   Scandinavian       10.38
3  2016     10   30  Sunday  morning    10    7   57            3  Hot chocolate        9.58
4  2016     10   30  Sunday  morning    10    7   57            3            Jam       10.08
	The busiest timeframes for transactions are as follows.
	-Hour: 11
	-Day of Week: Saturday
	-Period: afternoon
	The most profitable times for highest revenue are as follows.
	-Hour: 11
	-Day of Week: Saturday
	-Period: afternoon
	Most popular item is: Coffee (5471 units sold)
	Least popular item is: Adjustment (1 unit sold)
	Number of baristas required on each weekday given the limi